# Using SimEc for recommender systems
Tasks like product recommendation or drug-target interaction prediction essentially consist of having to predict missing entries in a large matrix containing pairwise relations, e.g., the user ratings of some items or whether or not a drug interacts with a certain protein. Besides the sparse matrix containing the pairwise relations, generally one can also construct some feature vectors for the items and users (drugs / proteins), e.g., based on textual descriptions. These can come in especially handy when predictions need to be made, e.g., for new items that did not receive any user ratings so far. In the following we will only talk about items and users but of course this extends to other problem setups as well like drug-target interaction prediction. We distingish between 3 tasks with increasing difficulty:
- **T1**: Predict missing ratings for exisiting items and users
- **T2a** and **T2b**: Predict ratings for new items and existing users (a) or new users and existing items (b)
- **T3**: Predict ratings for new items and new users

For tasks T2a/b and T3, feature vectors describing items and/or users are required. 

In this notebook, we discuss some methods which can be used to solve some or all of the above tasks. These include:
##### Baseline Methods
- **Predict average**: This is a no-brainer: simply fill all the missing values by averages. For example, an item rating from a user can be predicted based on the average rating the user usually gives (he might in general be more or less critical than other users) and the average rating the item got from other users (it might be better or worse than the average item) or for new items and users just predict the overall average rating (solves **T1, T2a/b, T3**).
- **SVD of the ratings matrix**: By factorizing the ratings matrix using singular value decomposition (SVD), one can compute a low rank approximation of the ratings matrix and use these approximate values as predictions for the missing values (solves **T1**). This can also be combined with the average ratings from above, i.e., the low rank approximation can be used to predict the residuals.
- **SVD + Regression**: Given some feature vectors for items or users and the low rank approximation of the ratings matrix computed above, using a regression model, the mapping from the items' feature vectors to their rating vectors can be learned (or respectively for users). This is an extension of the above method to additionally solve either **T2a** or **T2b**.
- **Regression/Classification model**: This approach is completely different from the so-called latent factor models discussed above. Here we train an ordinary regression or classification model (depending on the form of the pairwise data, e.g. continous ratings or binary interactions) by using as input the concatenation of the feature vectors of an item and a user and as the target their rating. This approach can be used to solve all tasks **T1, T2a/b, T3** provided corresponding feature vectors are available.

##### Similarity Encoder Models
- **Factorization of the ratings matrix using the identiy matrix as input**: By training a SimEc to factorize the ratings matrix using the identiy matrix as input, we can recreate the solution obtained with SVD (while better handeling missing values when computing the decomposition). Correspondingly, this only solves **T1**.
- **Factorization of the ratings matrix using feature vectors as input**: By using either item or user feature vectors as input when factorizing the ratings matrix, we can additionally solve **T2a** or **T2b**.
- **Train a second SimEc with feature vectors and fixed last layer weights**: After training, e.g., a SimEc with item feature vectors as input to decompose the ratings matrix, we can use this SimEc to compute the item embeddings $Y$. We can then construct a second SimEc, which uses user feature vectors as input to factorize the ratings matrix. However, here we fix the weights of the last layer by setting them to the transpose of the embedding matrix computed for the items. After this SimEc is trained, we can now use both SimEcs to compute item and user embeddings respectively and then compute the scalar product of the embedding vectors to predict the ratings. This approach can then also be used to predict the ratings given the feature vectors for new items and users, i.e., it can be used to solve all tasks **T1, T2a/b, T3**.

If the rating matrix contains explicit ratings (i.e. likes and dislikes), all available entries can be used to train the above models. If the pairwise relations in the matrix only represent implicit feedback or binary interactions (e.g. the user listens to music by certain artists, which means he likes them, but we don't know if he doesn't listen to other artists because he doesn't know them or because he doesn't like them), then we can use the given entries in the matrix as positive examples and additionally take a random sample of the missing entries and use them as negative examples. In the latter case, it might be more useful to use classification instead of regression models and also when training the SimEc it could be helpful to apply a non-linearity on the output before computing the error of the model.

In [1]:
import numpy as np
np.random.seed(28)
import matplotlib.pyplot as plt
import tensorflow as tf
tf.set_random_seed(28)
import keras

from simec import SimilarityEncoder

%matplotlib inline
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [ ]:
# load data

# matrix with pairwise data

# left and right input vectors
X1 = np.eye(n_input)
X2 = np.eye(n_output)

In [ ]:
X1 = np.eye(n_input)
X2 = np.eye(n_output)
mses1 = []
mses2 = []
mses3 = []
e_dims =  [2, 10, 25, 50, 75, 100, 250, 400, 500, 750, 1000]
for e_dim in e_dims:
    model = SimilarityEncoder(n_input, e_dim, n_output, opt=keras.optimizers.Adamax(lr=0.005 if e_dim <= 250 else l_rates[e_dim]))
    model.fit(X1, A, epochs=50)
    mse = msqe(A, model.predict(X1))
    mses1.append(mse)
    print "mse with %4i e_dim: %.8f" % (e_dim, mse)
    Y1 = model.transform(X1)
    model = SimilarityEncoder(n_output, e_dim, n_input, W_ll=Y1.T, opt=keras.optimizers.Adamax(lr=0.005 if e_dim <= 250 else l_rates[e_dim]))
    model.fit(X2, A.T, epochs=50)
    mse = msqe(A.T, model.predict(X2))
    mses2.append(mse)
    print "mse with %4i e_dim: %.8f" % (e_dim, mse)
    Y2 = model.transform(X2)
    # the dot product of both embeddings should also approximate A
    mse = msqe(A, np.dot(Y1, Y2.T))
    mses3.append(mse)
for i, e_dim in enumerate(e_dims):
    print "mse with %4i e_dim: %.8f / %.8f / %.8f" % (e_dim, mses1[i], mses2[i], mses3[i])